A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|--------------------------|----------------------------------------------------------------------------------|
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Importing libraries & files
import pandas as pd
import numpy as np

ds_jobs = pd.read_csv("data/customer_train.csv")

## Exploring dataset
* Exploring first rows
* Check memory usage
* Check dtypes

In [2]:
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [3]:
print(ds_jobs.memory_usage())
print("")
print(f"Total used memory: {np.sum(ds_jobs.memory_usage())}")

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

Total used memory: 2145824


In [4]:
ds_jobs.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                  int64
dtype: object

In [5]:
# Shape of the dataset
ds_jobs.shape

(19158, 14)

## Converting types
* Columns containing integer values should be converted to `int32`
* Columns containing float values should be converted to `float16`
* Columns containing nominal categorical data should be converted to `category`
* Columns containing ordinal categorical data should be converted to `category` with order

In [6]:
ds_jobs_clean = ds_jobs.copy()

# Creating a dictionary for key-value pairs for assigning correct value type to every column
column_datatype = {
    "student_id":"int32",
    "city":"category",
    "city_development_index":"float16",
    "gender":"category",
    "relevant_experience":"category",
    "enrolled_university":"category",
    "education_level":"category",
    "major_discipline":"category",
    "experience":"category",
    "company_size":"category",
    "company_type":"category",
    "last_new_job":"category",
    "training_hours":"int32",
    "job_change":"int32"
}

# Assigning every column with the correct type
for col, data_type in column_datatype.items():
    ds_jobs_clean[col] = ds_jobs_clean[col].astype(data_type)
    
# Verify if all columns are assigned the correct value type
ds_jobs_clean.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience       category
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                   int32
dtype: object

## Setting ordinal columns
* Create a dictionary containing the ordinal column and the category order
* Assign the appropriate order per column
* Print results for verification

In [7]:
# Create a dictionary containing columns and order of entries
ordinal_entries = {
    "enrolled_university":["no_enrollment", "Part time course", "Full time course"],
    "education_level":["Primary School", "High School", "Graduate", "Masters", "Phd"],
    "relevant_experience":["No relevant experience", "Has relevant experience"],
    "experience":["<1"] + list(map(str, range(1, 21))) + [">20"],
    "company_size":["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"],
    "last_new_job":["never", "1", "2", "3", "4", ">4"]
}

# Assigning every ordinal column with the correct ordered entries
for col, categories in ordinal_entries.items():
    # Set the appropriate ranking
    ds_jobs_clean[col] = ds_jobs_clean[col].cat.reorder_categories(
        new_categories = categories,
        ordered = True
    )
    # Print results for verification
    print(ds_jobs_clean[col])

0           no_enrollment
1           no_enrollment
2        Full time course
3                     NaN
4           no_enrollment
               ...       
19153       no_enrollment
19154       no_enrollment
19155       no_enrollment
19156       no_enrollment
19157       no_enrollment
Name: enrolled_university, Length: 19158, dtype: category
Categories (3, object): ['no_enrollment' < 'Part time course' < 'Full time course']
0              Graduate
1              Graduate
2              Graduate
3              Graduate
4               Masters
              ...      
19153          Graduate
19154          Graduate
19155          Graduate
19156       High School
19157    Primary School
Name: education_level, Length: 19158, dtype: category
Categories (5, object): ['Primary School' < 'High School' < 'Graduate' < 'Masters' < 'Phd']
0        Has relevant experience
1         No relevant experience
2         No relevant experience
3         No relevant experience
4        Has relevant experien

## Filtering the dataset for export
* Check memory usage of dataset
* Filter for: 
    1. Students with 10 years or more experience
    2. Experience at companies with at least 1000 employees
* Check if results are correct

In [8]:
# Check memory usage after cleaning
print(ds_jobs_clean.memory_usage())
print("")
print(f"Total used memory: {np.sum(ds_jobs_clean.memory_usage())}")
print(f"Assigning new data types saved: {np.sum(ds_jobs.memory_usage()) - np.sum(ds_jobs_clean.memory_usage())}")

Index                       128
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19282
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19890
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                76632
dtype: int64

Total used memory: 467428
Assigning new data types saved: 1678396


In [9]:
# Filter for:
#   Students with 10+ years experience
#   Company size of 1000+ employees
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean["experience"] >= "10") & (ds_jobs_clean["company_size"] >= "1000-4999")]

In [10]:
# Verify results
print(ds_jobs_clean.head())
print(f"Size of the dataset: {ds_jobs_clean.shape}")

    student_id      city  city_development_index  gender  \
9          699  city_103                0.919922     NaN   
12       25619   city_61                0.913086    Male   
31       22293  city_103                0.919922    Male   
34       26494   city_16                0.910156    Male   
40        2547  city_114                0.925781  Female   

        relevant_experience enrolled_university education_level  \
9   Has relevant experience       no_enrollment        Graduate   
12  Has relevant experience       no_enrollment        Graduate   
31  Has relevant experience    Part time course        Graduate   
34  Has relevant experience       no_enrollment        Graduate   
40  Has relevant experience    Full time course         Masters   

   major_discipline experience company_size   company_type last_new_job  \
9              STEM         17       10000+        Pvt Ltd           >4   
12             STEM        >20    1000-4999        Pvt Ltd            3   
31         